In [50]:
import paramiko
from getpass import getpass

# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "michael.pinelli"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationEsception:
    print("Authentication failed.")

In [51]:
blind_files_cmds = ['cat /etc/resolv.conf',
                    'cat /etc/motd',
                    'cat /etc/issue',
                    'cat /etc/passwd',
                    'cat /etc/shadow']
system_cmds = ['uname -a',
               'ps aux',
               'top -n 1 -d',
               'id',
               'uname -m']
netwroking_cmds = ['hostname -f',
                   'ip addr show',
                   'ip ro show',
                   'ifconfig -a',
                   'route -n']
user_acc_cmds = ['cat /etc/passwd',
                 'cat /etc/shadow',
                 'cat /etc/security/passwd',
                 'cat /etc/group',
                 'getent passwd']
obtain_info_cmds = ['ls -alh /home/*/',
                    'ls -alh /home/*/.ssh/',
                    'cat /home/*/.ssh/authorized_keys',
                    'cat /home/*/.ssh/known_hosts',
                    'cat /home/*/.hist']
cred_cmds = ['cat /home/*/.ssh/id*',
             'cat /tmp/krb5cc_*',
             'cat /tmp/krb5.keytab',
             'cat /home/*/.gnupg/secring.gpgs']
config_cmds = ["ls -aRl /etc/ * awk '$1 ~ /w.$/' * grep -v lrwx 2>/dev/nullte",
               "cat /etc/issue{,.net}",
               "cat /etc/master.passwd",
               "cat /etc/group",
               "cat /etc/hosts"]
determine_distro_cmds = ['uname -a',
                         'lsb_release -d',
                         'cat /etc/os-release',
                         'cat /etc/issue',
                         'cat /etc/*release']
installed_pkgs_cmds = ['dpkg -l',
                       'dpkg -l | grep -i “linux-image”',
                       'dpkg --get-selections']
pkgs_cources_cmds = ['cat /etc/apt/sources.list',
                     'ls -l /etc/yum.repos.d/',
                     'cat /etc/yum.conf']
find_imp_files_cmds = ['ls -dlR */',
                       'ls -alR | grep ^d',
                       'find /var -type d',
                       'ls -dl `find /var -type d`',
                       'ls -dl `find /var -type d` | grep -v root']
all_cmds = [blind_files_cmds, system_cmds, netwroking_cmds, user_acc_cmds, obtain_info_cmds, cred_cmds, config_cmds, determine_distro_cmds, installed_pkgs_cmds, pkgs_cources_cmds, find_imp_files_cmds]
# Run a command
#command = ['ps -ef', 'netstat -an --inet', 'last -adx', 'cut -d: -f1 /etc/passwd']
command = input("Which set of commands would you like to run? ")
for eachCMD in command:
    # Get the output from the command
    stdin, stdout, stderr = ssh.exec_command(command)
    
    stdin.write(thePass + "\n")

    # Get results from a stdout
    lines = stdout.readlines()

    # Convert the list to a string
    output = ''.join(lines)

    # Header output
    sepHeader = '' + '### BEGIN' + command + ' ###\n\n'

    # Footer
    sepFooter = '' + '### END' + command + ' ###\n\n'

    #Concatenate the header, output, and footer
    cmd_output = sepHeader + output + sepFooter

    # Save the cmd_output to a file
    with open('ssh_results.txt', 'w') as f:
        f.write(cmd_output)